# Natural Language Processing

# Retrieval-Augmented generation (RAG)

RAG is a technique for augmenting LLM knowledge with additional, often private or real-time, data.

LLMs can reason about wide-ranging topics, but their knowledge is limited to the public data up to a specific point in time that they were trained on. If you want to build AI applications that can reason about private data or data introduced after a model’s cutoff date, you need to augment the knowledge of the model with the specific information it needs.

<img src="../figures/RAG-process.png" >

Introducing `ChakyBot`, an innovative chatbot designed to assist Chaky (the instructor) and TA (Gun) in explaining the lesson of the NLP course to students. Leveraging LangChain technology, ChakyBot excels in retrieving information from documents, ensuring a seamless and efficient learning experience for students engaging with the NLP curriculum.

1. Prompt
2. Retrieval
3. Memory
4. Chain

In [ ]:
# #langchain library
# !pip install langchain==0.0.350
# #LLM
# !pip install accelerate==0.25.0
# !pip install transformers==4.36.2
# !pip install bitsandbytes==0.41.2
# #Text Embedding
# !pip install sentence-transformers==2.2.2
# !pip install InstructorEmbedding==1.0.1
# #vectorstore
# !pip install pymupdf==1.23.8
# !pip install faiss-gpu==1.7.2
# !pip install faiss-cpu==1.7.4

In [1]:
import os
import torch
# Set GPU device
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# os.environ['http_proxy']  = 'http://192.41.170.23:3128'
# os.environ['https_proxy'] = 'http://192.41.170.23:3128'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

## 1. Prompt

A set of instructions or input provided by a user to guide the model's response, helping it understand the context and generate relevant and coherent language-based output, such as answering questions, completing sentences, or engaging in a conversation.

In [2]:
from langchain import PromptTemplate

prompt_template = """
     You are a helpful assistant that answers questions about the person based on their personal documents.
    Use the following context to answer the question. If you don't know the answer, just say you don't know.
    Don't make things up.
    {context}
    Question: {question}
    Answer:
    """.strip()

PROMPT = PromptTemplate.from_template(
    template = prompt_template
)

PROMPT
#using str.format 
#The placeholder is defined using curly brackets: {} {}

PromptTemplate(input_variables=['context', 'question'], template="You are a helpful assistant that answers questions about the person based on their personal documents.\n    Use the following context to answer the question. If you don't know the answer, just say you don't know.\n    Don't make things up.\n    {context}\n    Question: {question}\n    Answer:")

In [3]:
PROMPT.format(
    context = "The person is a mechanical engineer and he has switched his career to data science and AI and is currently pursuing his masters in data science and AI.",
    question = "What is Machine Learning"
)

"You are a helpful assistant that answers questions about the person based on their personal documents.\n    Use the following context to answer the question. If you don't know the answer, just say you don't know.\n    Don't make things up.\n    The person is a mechanical engineer and he has switched his career to data science and AI and is currently pursuing his masters in data science and AI.\n    Question: What is Machine Learning\n    Answer:"

Note : [How to improve prompting (Zero-shot, Few-shot, Chain-of-Thought, etc.](https://github.com/chaklam-silpasuwanchai/Natural-Language-Processing/blob/main/Code/05%20-%20RAG/advance/cot-tot-prompting.ipynb)

## 2. Retrieval

1. `Document loaders` : Load documents from many different sources (HTML, PDF, code). 
2. `Document transformers` : One of the essential steps in document retrieval is breaking down a large document into smaller, relevant chunks to enhance the retrieval process.
3. `Text embedding models` : Embeddings capture the semantic meaning of the text, allowing you to quickly and efficiently find other pieces of text that are similar.
4. `Vector stores`: there has emerged a need for databases to support efficient storage and searching of these embeddings.
5. `Retrievers` : Once the data is in the database, you still need to retrieve it.

### 2.1 Document Loaders 
Use document loaders to load data from a source as Document's. A Document is a piece of text and associated metadata. For example, there are document loaders for loading a simple .txt file, for loading the text contents of any web page, or even for loading a transcript of a YouTube video.

[PDF Loader](https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf)

[Download Document](https://web.stanford.edu/~jurafsky/slp3/)

In [17]:
from langchain.document_loaders import PyMuPDFLoader

nlp_docs = './docs/aboutMe.pdf'

loader = PyMuPDFLoader(nlp_docs)
documents = loader.load()

In [18]:
# documents

In [19]:
len(documents)

6

In [20]:
documents[1]

Document(page_content='o This internal conflict surfaces when considering future paths, such as \npursuing a PhD or finding a job. \n \n3. Hobbies and Interests \n1. Music \no He plays the synthesizer and the ukulele. \no He is currently learning the guitar, though he mentions not being \nentirely successful yet. \no He generally performs for personal enjoyment because large \naudiences make him nervous or self-conscious. \n2. Chess \no He appreciates the strategic and mental stimulation the game offers. \n3. Badminton \no He played badminton up to Class 11. \no He competed in state-level badminton tournaments before \ndiscontinuing. \n \n4. Educational Background \n1. Schooling \no High School: St. Joseph’s College in Allahabad, Uttar Pradesh. \no Class 10 (ICSE Board): Scored 95%. \no Class 12 (CISCE/ICSE Board): Scored 92% in May 2018, earning full \nmarks in Computer Science. \n2. Undergraduate (B.Tech in Mechanical Engineering) \no Institution: SRM University, Chennai, India. \no 

### 2.2 Document Transformers

This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough

In [21]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 700,
    chunk_overlap = 100
)

doc = text_splitter.split_documents(documents)

In [22]:
doc[1]

Document(page_content='2. Personality and Traits \n1. Introversion and Social Circle \no He doesn’t talk much in large or unfamiliar groups. \no He has a tight-knit group of three friends he trusts with his life. \no Though introverted, he becomes highly engaging once comfortable. \n2. Calm Yet Challenge-Oriented \no He appreciates a peaceful environment. \no He enjoys taking on tasks that test his skills or resolve. \n3. Emotional Response to Encouragement vs. Doubt \no He works best with positive reinforcement. \no If someone he deeply respects suggests he cannot achieve \nsomething, he tends to feel demotivated. \no His mother’s encouragement often helps him regain confidence and \nmotivation.', metadata={'source': './docs/aboutMe.pdf', 'file_path': './docs/aboutMe.pdf', 'page': 0, 'total_pages': 6, 'format': 'PDF 1.7', 'title': '', 'author': 'Suryansh Srivastava', 'subject': '', 'keywords': '', 'creator': 'Microsoft® Word for Microsoft 365', 'producer': 'Microsoft® Word for Microso

In [23]:
len(doc)

14

### 2.3 Text Embedding Models
Embeddings create a vector representation of a piece of text. This is useful because it means we can think about text in the vector space, and do things like semantic search where we look for pieces of text that are most similar in the vector space.

*Note* Instructor Model : [Huggingface](gingface.co/hkunlp/instructor-base) | [Paper](https://arxiv.org/abs/2212.09741)

In [31]:
import torch
from langchain.embeddings import HuggingFaceInstructEmbeddings

model_name = 'hkunlp/instructor-base'

embedding_model = HuggingFaceInstructEmbeddings(
    model_name = model_name,
    model_kwargs = {"device" : device}
)

load INSTRUCTOR_Transformer
max_seq_length  512


### 2.4 Vector Stores

One of the most common ways to store and search over unstructured data is to embed it and store the resulting embedding vectors, and then at query time to embed the unstructured query and retrieve the embedding vectors that are 'most similar' to the embedded query. A vector store takes care of storing embedded data and performing vector search for you.

In [32]:
#locate vectorstore
vector_path = '../vector-store'
if not os.path.exists(vector_path):
    os.makedirs(vector_path)
    print('create path done')

In [33]:
#save vector locally
from langchain.vectorstores import FAISS

vectordb = FAISS.from_documents(
    documents = doc,
    embedding = embedding_model
)

db_file_name = 'nlp_stanford'

vectordb.save_local(
    folder_path = os.path.join(vector_path, db_file_name),
    index_name = 'nlp' #default index
)

### 2.5 retrievers
A retriever is an interface that returns documents given an unstructured query. It is more general than a vector store. A retriever does not need to be able to store documents, only to return (or retrieve) them. Vector stores can be used as the backbone of a retriever, but there are other types of retrievers as well.

In [34]:
#calling vector from local
vector_path = '../vector-store'
db_file_name = 'nlp_stanford'

from langchain.vectorstores import FAISS

vectordb = FAISS.load_local(
    folder_path = os.path.join(vector_path, db_file_name),
    embeddings = embedding_model,
    index_name = 'nlp' #default index
)   

In [35]:
#ready to use
retriever = vectordb.as_retriever()

In [36]:
retriever.get_relevant_documents("How old are you")


[Document(page_content='1. Basic Personal Information \n• \nFull Name: Suryansh Srivastava \n• \nDate & Time of Birth: April 22, 2000, at 7:45 PM \n• \nPlace of Birth: Allahabad (Prayagraj), Uttar Pradesh, India \n• \nFamily Details:  \no Father: Mr. Ajal Krishna \no Mother: Mrs. Deepti Srivastava \no Younger Sister: Aditi Srivastava (6 years younger) \nHe describes himself as someone who loves challenges but also values a calm life. \nHe is introverted and not very social, generally having only three close friends \nwhom he trusts deeply. When he does open up to people, he becomes very fun and \nlively, showing an enthusiastic and entertaining side. \n \n2. Personality and Traits \n1. Introversion and Social Circle', metadata={'source': './docs/aboutMe.pdf', 'file_path': './docs/aboutMe.pdf', 'page': 0, 'total_pages': 6, 'format': 'PDF 1.7', 'title': '', 'author': 'Suryansh Srivastava', 'subject': '', 'keywords': '', 'creator': 'Microsoft® Word for Microsoft 365', 'producer': 'Microso

In [37]:
retriever.get_relevant_documents("What is your highest level of education")

[Document(page_content='discontinuing. \n \n4. Educational Background \n1. Schooling \no High School: St. Joseph’s College in Allahabad, Uttar Pradesh. \no Class 10 (ICSE Board): Scored 95%. \no Class 12 (CISCE/ICSE Board): Scored 92% in May 2018, earning full \nmarks in Computer Science. \n2. Undergraduate (B.Tech in Mechanical Engineering) \no Institution: SRM University, Chennai, India. \no Graduation Date: June 2022. \no CGPA: 9.54. \no Scholarship: Received one for the first three years due to academic \nperformance. \no Final-Year Project: Focused on a “cool energy storage system.”', metadata={'source': './docs/aboutMe.pdf', 'file_path': './docs/aboutMe.pdf', 'page': 1, 'total_pages': 6, 'format': 'PDF 1.7', 'title': '', 'author': 'Suryansh Srivastava', 'subject': '', 'keywords': '', 'creator': 'Microsoft® Word for Microsoft 365', 'producer': 'Microsoft® Word for Microsoft 365', 'creationDate': "D:20250315234941+07'00'", 'modDate': "D:20250315234941+07'00'", 'trapped': ''}),
 Doc

## 3. Memory

One of the core utility classes underpinning most (if not all) memory modules is the ChatMessageHistory class. This is a super lightweight wrapper that provides convenience methods for saving HumanMessages, AIMessages, and then fetching them all.

You may want to use this class directly if you are managing memory outside of a chain.


In [38]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()
history

ChatMessageHistory(messages=[])

In [39]:
history.add_user_message('hi')
history.add_ai_message('Whats up?')
history.add_user_message('How are you')
history.add_ai_message('I\'m quite good. How about you?')

In [40]:
history

ChatMessageHistory(messages=[HumanMessage(content='hi'), AIMessage(content='Whats up?'), HumanMessage(content='How are you'), AIMessage(content="I'm quite good. How about you?")])

### 3.1 Memory types

There are many different types of memory. Each has their own parameters, their own return types, and is useful in different scenarios. 
- Converstaion Buffer
- Converstaion Buffer Window

What variables get returned from memory

Before going into the chain, various variables are read from memory. These have specific names which need to align with the variables the chain expects. You can see what these variables are by calling memory.load_memory_variables({}). Note that the empty dictionary that we pass in is just a placeholder for real variables. If the memory type you are using is dependent upon the input variables, you may need to pass some in.

In this case, you can see that load_memory_variables returns a single key, history. This means that your chain (and likely your prompt) should expect an input named history. You can usually control this variable through parameters on the memory class. For example, if you want the memory variables to be returned in the key chat_history you can do:

#### Converstaion Buffer
This memory allows for storing messages and then extracts the messages in a variable.

In [41]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': "Human: hi\nAI: What's up?\nHuman: How are you?\nAI: I'm quite good. How about you?"}

In [42]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages = True)
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': [HumanMessage(content='hi'),
  AIMessage(content="What's up?"),
  HumanMessage(content='How are you?'),
  AIMessage(content="I'm quite good. How about you?")]}

#### Conversation Buffer Window
- it keeps a list of the interactions of the conversation over time. 
- it only uses the last K interactions. 
- it can be useful for keeping a sliding window of the most recent interactions, so the buffer does not get too large.

In [43]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)
memory.save_context({'input':'hi'}, {'output':'What\'s up?'})
memory.save_context({"input":'How are you?'},{'output': 'I\'m quite good. How about you?'})
memory.load_memory_variables({})

{'history': "Human: How are you?\nAI: I'm quite good. How about you?"}

## 4. Chain

Using an LLM in isolation is fine for simple applications, but more complex applications require chaining LLMs - either with each other or with other components.

An `LLMChain` is a simple chain that adds some functionality around language models.
- it consists of a `PromptTemplate` and a `LM` (either an LLM or chat model).
- it formats the prompt template using the input key values provided (and also memory key values, if available), 
- it passes the formatted string to LLM and returns the LLM output.

Note : [Download Fastchat Model Here](https://huggingface.co/lmsys/fastchat-t5-3b-v1.0)

In [45]:
%cd ./models
!git clone https://huggingface.co/lmsys/fastchat-t5-3b-v1.0

d:\ProgLib\AIT-NLP-Assignment6\notebooks\models


Cloning into 'fastchat-t5-3b-v1.0'...
Updating files:  70% (7/10)
Updating files:  80% (8/10)
Updating files:  90% (9/10)
Updating files: 100% (10/10)
Updating files: 100% (10/10), done.


In [ ]:
from transformers import AutoTokenizer, pipeline, AutoModelForSeq2SeqLM
from transformers import BitsAndBytesConfig
from langchain import HuggingFacePipeline
import torch

model_id = '../models/fastchat-t5-3b-v1.0/'

tokenizer = AutoTokenizer.from_pretrained(
    model_id)

tokenizer.pad_token_id = tokenizer.eos_token_id

bitsandbyte_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.float16,
    bnb_4bit_use_double_quant = True
)

model = AutoModelForSeq2SeqLM.from_pretrained(
    model_id,
    quantization_config = bitsandbyte_config, #caution Nvidia
    device_map = 'auto',
    load_in_8bit = True
)

pipe = pipeline(
    task="text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens = 256,
    model_kwargs = {
        "temperature" : 0,
        "repetition_penalty": 1.5
    }
)

llm = HuggingFacePipeline(pipeline = pipe)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


ImportError: 
T5Converter requires the protobuf library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/protocolbuffers/protobuf/tree/master/python#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.


### [Class ConversationalRetrievalChain](https://api.python.langchain.com/en/latest/_modules/langchain/chains/conversational_retrieval/base.html#ConversationalRetrievalChain)

- `retriever` : Retriever to use to fetch documents.

- `combine_docs_chain` : The chain used to combine any retrieved documents.

- `question_generator`: The chain used to generate a new question for the sake of retrieval. This chain will take in the current question (with variable question) and any chat history (with variable chat_history) and will produce a new standalone question to be used later on.

- `return_source_documents` : Return the retrieved source documents as part of the final result.

- `get_chat_history` : An optional function to get a string of the chat history. If None is provided, will use a default.

- `return_generated_question` : Return the generated question as part of the final result.

- `response_if_no_docs_found` : If specified, the chain will return a fixed response if no docs are found for the question.


`question_generator`

In [ ]:
from langchain.chains import LLMChain
from langchain.chains.conversational_retrieval.prompts import CONDENSE_QUESTION_PROMPT
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import ConversationalRetrievalChain

In [ ]:
CONDENSE_QUESTION_PROMPT

In [ ]:
question_generator = LLMChain(
    llm = llm,
    prompt = CONDENSE_QUESTION_PROMPT,
    verbose = True
)

In [ ]:
query = 'Comparing both of them'
chat_history = "Human:What is Machine Learning\nAI:\nHuman:What is Deep Learning\nAI:"

question_generator({'chat_history' : chat_history, "question" : query})

`combine_docs_chain`

In [ ]:
doc_chain = load_qa_chain(
    llm = llm,
    chain_type = 'stuff',
    prompt = PROMPT,
    verbose = True
)
doc_chain

In [ ]:
query = "What is Transformers?"
input_document = retriever.get_relevant_documents(query)

doc_chain({'input_documents':input_document, 'question':query})

In [ ]:
memory = ConversationBufferWindowMemory(
    k=3, 
    memory_key = "chat_history",
    return_messages = True,
    output_key = 'answer'
)

chain = ConversationalRetrievalChain(
    retriever=retriever,
    question_generator=question_generator,
    combine_docs_chain=doc_chain,
    return_source_documents=True,
    memory=memory,
    verbose=True,
    get_chat_history=lambda h : h
)
chain

## 5. Chatbot

In [ ]:
prompt_question = "Who are you by the way?"
answer = chain({"question":prompt_question})
answer

In [ ]:
prompt_question = "What is the Transformers?"
answer = chain({"question":prompt_question})
answer

In [ ]:
prompt_question = "Is it a statistical model?"
answer = chain({"question":prompt_question})
answer